<a href="https://colab.research.google.com/github/jeffvun/Machine-Learning-Labs/blob/main/Bayesian_Stats_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
install.packages('rstan')
install.packages('tidyverse')

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘checkmate’, ‘matrixStats’, ‘StanHeaders’, ‘inline’, ‘gridExtra’, ‘Rcpp’, ‘RcppParallel’, ‘loo’, ‘QuickJSR’, ‘RcppEigen’, ‘BH’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [7]:
# Load Required Packages
library(rstan)
library(tidyverse)
library(haven)

In [9]:
# Data Import
hospital_data <- read_dta("/content/hospital_at_home.dta")
summary(hospital_data)

      age            age_z              sex          intervention   
 Min.   :15.91   Min.   :-3.5455   Min.   :0.0000   Min.   :0.0000  
 1st Qu.:57.25   1st Qu.:-0.7053   1st Qu.:0.0000   1st Qu.:0.0000  
 Median :69.80   Median : 0.1565   Median :0.0000   Median :0.0000  
 Mean   :67.52   Mean   : 0.0000   Mean   :0.4815   Mean   :0.3968  
 3rd Qu.:78.50   3rd Qu.: 0.7547   3rd Qu.:1.0000   3rd Qu.:1.0000  
 Max.   :95.30   Max.   : 1.9085   Max.   :1.0000   Max.   :1.0000  
 length_of_stay   
 Min.   :  1.071  
 1st Qu.: 11.494  
 Median : 19.367  
 Mean   : 26.008  
 3rd Qu.: 32.103  
 Max.   :172.646  

In [10]:
# Model Specification
model_code <- "
data {
  int<lower=0> N;
  vector[N] age_z;
  int<lower=0, upper=1> male[N];
  int<lower=0, upper=1> intervention[N];
  vector[N] length_of_stay;
}
parameters {
  real beta0;
  real beta1;
  real beta2;
  real delta;
  real<lower=0> sigma;
}
model {
  beta0 ~ normal(0, 1);
  beta1 ~ normal(0, 1);
  beta2 ~ normal(0, 1);
  delta ~ normal(0, 1);
  sigma ~ cauchy(0, 1);

  for (i in 1:N) {
    length_of_stay[i] ~ normal(beta0 + beta1 * age_z[i] + beta2 * male[i] + delta * intervention[i], sigma);
  }
}
"


In [11]:
# Prepare data for Stan
stan_data <- list(
  N = nrow(hospital_data),
  age_z = hospital_data$age_z,
  male = hospital_data$sex,
  intervention = hospital_data$intervention,
  length_of_stay = hospital_data$length_of_stay
)


In [12]:
# Bayesian estimation
stan_model <- stan_model(model_code = model_code)
stan_fit <- sampling(stan_model, data = stan_data, chains = 4, iter = 1000)



SAMPLING FOR MODEL 'anon_model' NOW (CHAIN 1).
Chain 1: 
Chain 1: Gradient evaluation took 0.000609 seconds
Chain 1: 1000 transitions using 10 leapfrog steps per transition would take 6.09 seconds.
Chain 1: Adjust your expectations accordingly!
Chain 1: 
Chain 1: 
Chain 1: Iteration:   1 / 1000 [  0%]  (Warmup)
Chain 1: Iteration: 100 / 1000 [ 10%]  (Warmup)
Chain 1: Iteration: 200 / 1000 [ 20%]  (Warmup)
Chain 1: Iteration: 300 / 1000 [ 30%]  (Warmup)
Chain 1: Iteration: 400 / 1000 [ 40%]  (Warmup)
Chain 1: Iteration: 500 / 1000 [ 50%]  (Warmup)
Chain 1: Iteration: 501 / 1000 [ 50%]  (Sampling)
Chain 1: Iteration: 600 / 1000 [ 60%]  (Sampling)
Chain 1: Iteration: 700 / 1000 [ 70%]  (Sampling)
Chain 1: Iteration: 800 / 1000 [ 80%]  (Sampling)
Chain 1: Iteration: 900 / 1000 [ 90%]  (Sampling)
Chain 1: Iteration: 1000 / 1000 [100%]  (Sampling)
Chain 1: 
Chain 1:  Elapsed Time: 2.584 seconds (Warm-up)
Chain 1:                0.879 seconds (Sampling)
Chain 1:                3.463 seconds 

In [13]:
# MCMC Diagnostics
print(stan_fit)


Inference for Stan model: anon_model.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

          mean se_mean   sd     2.5%      25%      50%      75%    97.5% n_eff
beta0    10.24    0.02 0.90     8.42     9.65    10.25    10.85    11.97  2004
beta1     2.51    0.02 0.76     1.04     2.00     2.51     3.01     4.00  2375
beta2     4.25    0.02 0.87     2.61     3.66     4.22     4.82     5.93  2441
delta     2.26    0.02 0.89     0.51     1.65     2.27     2.88     3.98  2495
sigma    26.77    0.02 0.94    24.94    26.13    26.75    27.40    28.63  2020
lp__  -2218.94    0.05 1.58 -2222.89 -2219.76 -2218.66 -2217.75 -2216.79   832
      Rhat
beta0    1
beta1    1
beta2    1
delta    1
sigma    1
lp__     1

Samples were drawn using NUTS(diag_e) at Sun Jan 28 12:11:42 2024.
For each parameter, n_eff is a crude measure of effective sample size,
and Rhat is the potential scale reduction factor on split chains (at 
converg

In [14]:
# Model Parameter Estimates
summary(stan_fit)


,mean,se_mean,sd,2.5%,25%,50%,75%,97.5%,n_eff,Rhat
beta0,10.235222,0.02015633,0.9022194,8.4244386,9.647820,10.252732,10.850728,11.968107,2003.556,1.0018286
beta1,2.511851,0.01549277,0.7550406,1.0385291,1.997228,2.508532,3.008662,4.000576,2375.103,1.0007874
beta2,4.245888,0.01758689,0.8689105,2.6119714,3.660297,4.220993,4.817763,5.925929,2441.023,1.0017301
delta,2.258573,0.01790077,0.8941934,0.5059178,1.652743,2.265484,2.883105,3.982564,2495.281,0.9989951
sigma,26.766514,0.02096039,0.9420139,24.9446340,26.130370,26.750633,27.398413,28.633035,2019.835,1.0003281
lp__,-2218.941586,0.05479138,1.5802399,-2222.8876106,-2219.760089,-2218.657112,-2217.749868,-2216.793466,831.805,1.0004630


In [26]:
# Calculate per-patient net costs
cost_per_bed_day <- 350
per_patient_program_cost <- 20000
posterior_samples <- as.data.frame(stan_fit)
mean_difference <- mean(posterior_samples$sigma)

probability_cost_saving <- pnorm(0, mean_difference * cost_per_bed_day - per_patient_program_cost, sd(posterior_samples$sigma))
probability_cost_saving


[1] 1